# Analysing the daily reports

In [ ]:
import pandas as pd
from glob import glob
import numpy as np
import panel as pn
pn.extension("tabulator")

import hvplot.pandas
import plotly.express as px
import plotly.graph_objects as go

import json


In [ ]:
import plotly.io as pio
# pio.renderers.default = "browser"

In [ ]:
def wrangler(file):
    # Read csv file
    df = pd.read_csv(file)

    # Changes column name "Case-Fatality_Ratio" to "Case_Fatality_Ratio"
    if "Case-Fatality_Ratio" in df.columns:
        df.rename(columns={"Case-Fatality_Ratio": "Case_Fatality_Ratio"}, inplace=True)

    # Changes column name "Incidence_Rate" to "Incident_Rate"
    if "Incidence_Rate" in df.columns:
        df.rename(columns={"Incidence_Rate": "Incident_Rate"}, inplace=True)
        
    # Europe Mask
    european_countries = [
    "Albania",
    "Andorra",
    "Austria",
    "Belarus",
    "Belgium",
    "Bosnia and Herzegovina",
    "Bulgaria",
    "Croatia",
    "Cyprus",
    "Czech Republic",
    "Denmark",
    "Estonia",
    "Finland",
    "France",
    "Germany",
    "Greece",
    "Hungary",
    "Iceland",
    "Ireland",
    "Italy",
    "Kosovo",
    "Latvia",
    "Liechtenstein",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Moldova",
    "Monaco",
    "Montenegro",
    "Netherlands",
    "North Macedonia",
    "Norway",
    "Poland",
    "Portugal",
    "Romania",
    "Russia",
    "San Marino",
    "Serbia",
    "Slovakia",
    "Slovenia",
    "Spain",
    "Sweden",
    "Switzerland",
    "Ukraine",
    "United Kingdom",
    "Vatican City (Holy See)",]

    
    
    # Drop Incident_Rate column
    if "Incident_Rate" in df.columns:
        df.drop(columns="Incident_Rate", inplace=True)

    # Checks if dataframe start with "FIPS" and "Admin2" and "Province_State" columns to eliminate differently structured dataset
    if "FIPS" and "Admin2" and "Province_State" in df.columns:
        
        euro_mask = df["Country_Region"].isin(european_countries)
        df = df[euro_mask]

        # Drop columns "FIPS", "Admin2", "Province_State" 
        df.drop(columns=["FIPS", "Admin2", "Province_State"], inplace=True)

        # Drop rows with no coordinate data
        df.dropna(subset="Lat", inplace=True)

        # Checks if column Case_Fatality_Ratio exists then populate the missing cells
        if "Case_Fatality_Ratio" in df.columns:
            df.drop(columns="Case_Fatality_Ratio", inplace=True)
        
        # Converts Last_Update column to datetime and rest time to 00:00
        df["Last_Update"] = pd.to_datetime(pd.to_datetime(df["Last_Update"]).dt.date)

        # Returns cleaned dataset
        return df

In [ ]:
# Appends all datasets in the folder to a list
files = glob("./COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/*.csv")

In [ ]:
# Appends cleaned dataset to a list
frames = []
for file in files:
    # print(file)
    df = wrangler(file)
    frames.append(df)

In [ ]:
# Concatenate dataframes
df = pd.concat(frames, ignore_index=True)
df.shape

In [ ]:
# Extract required columns and group data based on country and last-update
df_new = df.groupby(["Country_Region", "Last_Update"])[["Confirmed", "Deaths", "Recovered"]].max().reset_index()

In [ ]:
df_new.describe()

In [ ]:
boxplot = df_new.hvplot.box(y="Confirmed", by="Country_Region", height=400, width=1000, legend=False).opts(xrotation=90)
boxplot

In [ ]:
# Get data on country population
pop_data = "./COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv"
pop_df = pd.read_csv(pop_data)
pop_df = pop_df[["Country_Region", "Population"]]

In [ ]:
#Get data on population density
popd = pd.read_csv("population-density.csv")
popd = popd[popd["Year"] == 2023][["Entity", "Population density"]]

In [ ]:
popd.rename(columns={"Entity": "Country_Region", "Population density": "population_density"}, inplace=True)

In [ ]:
# Merge population data with covid data
df_new = pd.merge(df_new, pop_df, on="Country_Region")

In [ ]:
df_new = pd.merge(df_new, popd, on="Country_Region")

In [ ]:
df_new

In [ ]:
idf = df_new.interactive()

In [ ]:
datetime_picker = pn.widgets.DatetimePicker(name='Datetime Picker', value=df_new["Last_Update"].min(), start=df_new["Last_Update"].min(), end=df_new["Last_Update"].max())

datetime_picker

In [ ]:
# Radio buttons for Confirmed Cases and Deaths
yaxis_cases = pn.widgets.RadioButtonGroup(name="Y-Axis", options=["Confirmed", "Deaths", "Recovered"], button_type="success")

In [ ]:
covid_pipeline = (
    idf[(idf["Last_Update"] <= datetime_picker)]
    .groupby(["Last_Update", "Country_Region",])[yaxis_cases].mean()
    .to_frame()
    .reset_index()
    .sort_values(by="Last_Update")
    .reset_index(drop=True)
)

In [ ]:
covid_plot = covid_pipeline.hvplot(x = "Last_Update", by="Country_Region", y = yaxis_cases, line_width=2, title="Number of Cases Over Time")
covid_plot

In [ ]:
covid_table = covid_pipeline.pipe(pn.widgets.Tabulator, pagination="remote", page_size = 10, sizing_mode="scale_both")
covid_table

In [ ]:
covid_popd_pipeline = (
    idf[(idf["Last_Update"] == datetime_picker)]
    .groupby(["Last_Update", "Country_Region"])[[yaxis_cases, "population_density"]].max()
    .reset_index()
    .sort_values(by="Last_Update")
    .reset_index(drop=True)
)

In [ ]:
covid_popd_scatterplot = covid_popd_pipeline.hvplot(x=yaxis_cases, y="population_density", by="Country_Region", size=80, kind="scatter", alpha=0.7, legend=False, height=500, width=500)

covid_popd_scatterplot

In [ ]:
covid_pop_pipeline = (
    idf[(idf["Last_Update"] == datetime_picker)]
    .groupby(["Last_Update", "Country_Region"])[[yaxis_cases, "Population"]].max()
    .reset_index()
    .sort_values(by="Last_Update")
    .reset_index(drop=True)
)

In [ ]:
covid_pop_scatterplot = covid_pop_pipeline.hvplot(x=yaxis_cases, y="Population", by="Country_Region", size=80, kind="scatter", alpha=0.7, legend=False, height=500, width=500)

covid_pop_scatterplot

In [ ]:
europe = json.load(open("maps/europe.geojson"))

In [ ]:
country_id = {}
for feature in europe['features']:
    feature['id'] = feature['properties']['NAME']
    country_id[feature['properties']["NAME"]] = feature['id']

In [ ]:
df_new["year"] = pd.to_datetime(df_new["Last_Update"]).dt.year

In [ ]:
df_new["month"] = pd.to_datetime(df_new["Last_Update"]).dt.month

In [ ]:
df_new = df_new.sort_values("Last_Update")

In [ ]:
df_new

In [ ]:
df_new = df_new.groupby(["Country_Region","year", "month"]).max().reset_index().sort_values(["year", "month"])

In [ ]:
df_new

In [ ]:
# df_new['id'] = df_new['AREA'].apply(lambda x: country_id[x])
df_new['id'] = [country_id[x['Country_Region']] if x["Country_Region"] in country_id.keys() else None for _, x in df_new.iterrows()]

In [ ]:
df_new = df_new[df_new["Country_Region"].isin(country_id.keys())]

In [ ]:
df_new = df_new.sort_values(['month','year'])

In [ ]:
fig = px.choropleth(
    df_new, 
    locations="Country_Region", 
    geojson=europe, 
    range_color=(df_new['Confirmed'].min(), df_new['Confirmed'].max()), 
    color="Confirmed", 
#     height=1000,
#     width=1500,
    animation_frame="month",
    animation_group="Country_Region",
    scope="europe",
    color_continuous_scale=["#fffed3", "#FF0000"],
    title=f"Number of Confirmed COVID Cases in Europe",
    facet_col="year",
    facet_col_wrap=2,

)

In [ ]:
fig.update_geos(fitbounds='locations')
fig.update_layout(
    autosize=True
)
# fig.update_layout(margin={"r": 10, "t": 10, "l": 10, "b": 10})
# fig.show()

In [ ]:
# Layout using template
template = pn.template.FastListTemplate(
    title = "Insight on COVID-19 Cases in Europe",
    sidebar = [
        pn.pane.Markdown("# Relationship Between Population Size and Reported Number of COVID-19 Cases in Europe"),
        pn.pane.Markdown("#### Our comprehensive data analysis project on COVID-19 cases across Europe has revealed insightful findings, particularly concerning the relationship between a country's population size and its reported number of COVID-19 cases. Notably, we found a positive correlation between these two variables. This suggests that countries with larger populations have generally reported a higher number of COVID-19 cases. For instance, heavily populated countries like Germany, the United Kingdom, and France have consistently reported higher case numbers in comparison to smaller nations such as Luxembourg and Malta. However, it's important to emphasize that this analysis does not take into account other potentially influential factors such as testing rates, public health policy measures, or population density. Consequently, while a positive correlation was observed, further analysis would be needed to ascertain the cause-and-effect relationship and account for these additional factors. Nonetheless, this study provides a useful starting point for understanding the broad impact of the COVID-19 pandemic across Europe."),
        pn.pane.PNG("europe-corona-covid.png", sizing_mode="scale_both"),
        pn.pane.Markdown("## Settings"),
        datetime_picker
    ],
    main=[
        pn.Row(pn.Column(yaxis_cases, covid_plot.panel(sizing_mode="scale_both"), margin=(0,25)), covid_table.panel(sizing_mode="scale_both")),
        pn.Row(pn.Column(covid_pop_scatterplot.panel(sizing_mode="scale_both"), margin=(0,25)),
              pn.Column(covid_popd_scatterplot.panel(sizing_mode="scale_both"), margin=(0,25))),
        pn.Row(pn.pane.Plotly(fig, sizing_mode='stretch_width'), height=1000)
    ],
    accent_base_color="#88d8b0",
    header_background="#88d8b0"
)

template.servable();

# Interactive Dashboard for this Project

Run the code below in your terminal to open the dashboard:

*panel serve Analysis_of_Europe_COVID-19_Cases.ipynb*


![Project_Screenshot](https://github.com/itsveence/Analysis-of-Europe-COVID-19-Cases/assets/20642373/a5f4434d-5cdb-49e2-942b-21d5bbde08df)